<a href="https://colab.research.google.com/github/raunak-sood2003/CS224w/blob/main/Process_poitional_encoding_Mangrove_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Mount the drive and set the directory

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment2/'
FOLDERNAME = 'CS224W ML on Graphs/2025/Project/Mangrove-data/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# Change directory to current path
%cd /content/drive/My\ Drive/$FOLDERNAME/
!pwd

/content/drive/My Drive/CS224W ML on Graphs/2025/Project/Mangrove-data
/content/drive/My Drive/CS224W ML on Graphs/2025/Project/Mangrove-data


### Download the original pkl data

In [ ]:
# we're going to be using Pytorch, the most widely used framework for ML research right now.
# Pytorch is already installed so all we have to worry about is that everything else we do is compatible with our version of Pytorch
import torch
import os
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.9.0+cu126


In [ ]:
#here we load the libraries needed for doing ML on graphs, pytorch geometric, and a little package that can help with making our training faster
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
!pip install accelerate

Looking in links: https://pytorch-geometric.com/whl/torch-2.9.0+cu126.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp312-cp312-linux_x86_64.whl size=3039810 sha256=4cb25ff2cf808b8f24153ff02a35117d51be162f0ab9def58ce45f27103f3451
  Stored in directory: /root/.cache/pip/wheels/71/fa/21/bd1d78ce1629aec4ecc924a63b82f6949dda484b6321eac6f2
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp312-cp312-linux_x86_64.whl size=3857026 sha256=6d5a55

In [ ]:
import pickle, time, os
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import random
import string
from tqdm import tqdm
from accelerate import Accelerator
from datetime import date

import torch_geometric as tg
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

from IPython import display
import matplotlib as mpl

font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 16}

mpl.rc('font', **font)

Only need to do this step once, otherwise it's in the '/content/drive/My Drive/CS224W ML on Graphs/2025/Project/Mangrove-data' directory

In [ ]:
!gdown 'https://drive.google.com/u/0/uc?id=192_gOXbEAhp4BSARobCsmJP2f98LsbGG' # 25% of full data file, 1.56 GB
!gdown 'https://drive.google.com/u/0/uc?id=1FjyePBL8FNfTudRDMffHmamnBVq7EKAW' # configuration file
!gdown 'https://drive.google.com/u/0/uc?id=17U1qqOvLj2J8TK0LTFUeMJcR26xLq5bQ' # model
!gdown 'https://drive.google.com/u/0/uc?id=12MCG8u6TYNSD-Y5exWe1yvu7rYUHQCyR' # results file

Downloading...
From (original): https://drive.google.com/u/0/uc?id=192_gOXbEAhp4BSARobCsmJP2f98LsbGG
From (redirected): https://drive.google.com/uc?id=192_gOXbEAhp4BSARobCsmJP2f98LsbGG&confirm=t&uuid=00b6047e-c797-41a3-bf40-781573be377f
To: /content/drive/My Drive/CS224W ML on Graphs/2025/Project/Mangrove-data/data25.pkl
 47% 790M/1.68G [00:12<00:10, 86.9MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gdown/__main__.py", line 172, in main
    download(
  File "/usr/local/lib/python3.12/dist-packages/gdown/download.py", line 369, in download
    f.write(chunk)
KeyboardInterrupt
 47% 791M/1.68G [00:12<00:13, 63.4MB/s]
^C
Downloading...
From: https://drive.google.com/u/0/uc?id=1FjyePBL8FNfTudRDMffHmamnBVq7EKAW
To: /content/drive/MyDrive/CS224W ML on Graphs/2025/Project/Mangrove-data/construct_dict.pkl
100% 852/852 [00:00<00:00, 3.79MB/s]
Downloading...
From:

In [ ]:
data = pickle.load(open('data25.pkl', 'rb'))
config = pickle.load(open('construct_dict.pkl', 'rb'))

### Check Model

In [ ]:
print("")
print("number of graphs: ", len(data))
print('==============================================================')
idx = 4
one_graph = data[idx]  # Get the first graph object.

# Gather some statistics about the graph.
print("graph at index: ", idx)
print(one_graph)
print(f'Number of nodes: {one_graph.num_nodes}')
print(f'Number of edges: {one_graph.num_edges}')
print(f'Average node degree: {(one_graph.num_edges) / one_graph.num_nodes:.2f}')
print(f'Contains isolated nodes: {one_graph.has_isolated_nodes()}')
print(f'Contains self-loops: {one_graph.has_self_loops()}')
print(f'Is undirected: {one_graph.is_undirected()}')


number of graphs:  27085
graph at index:  4
Data(x=[10230, 43], edge_index=[2, 10229], edge_attr=[10229], y=[41])
Number of nodes: 10230
Number of edges: 10229
Average node degree: 1.00
Contains isolated nodes: False
Contains self-loops: False
Is undirected: False


### Compute the Laplacian
You can choose which Laplacian you want:

- Combinatorial: $L=D−A$
- Symmetric normalized: $L_{sym} =I−D^{−1/2} A D^{−1/2}$
- Random-walk normalized: $L_{rw} = I − D^{−1} A$

In [ ]:
from torch_geometric.utils import get_laplacian, to_dense_adj, to_scipy_sparse_matrix, to_undirected
from scipy.sparse.linalg import eigsh  # sparse symmetric eigensolver

def combinatorial_laplacian(data):
    # data.edge_index is shape [2, num_edges]
    edge_index = data.edge_index

    # Returns edges and weights defining L = D - A
    lap_edge_index, lap_edge_weight = get_laplacian(edge_index, normalization=None)

    # Convert sparse representation to dense matrix
    L = to_dense_adj(lap_edge_index, edge_attr=lap_edge_weight,
                     max_num_nodes=data.num_nodes).squeeze(0)
    return L


def normalized_laplacian(data):
    edge_index = data.edge_index

    # Returns edges/weights for L_sym = I - D^{-1/2} A D^{-1/2}
    lap_edge_index, lap_edge_weight = get_laplacian(edge_index, normalization='sym')

    L = to_dense_adj(lap_edge_index, edge_attr=lap_edge_weight,
                     max_num_nodes=data.num_nodes).squeeze(0)
    return L


def random_walk_laplacian(data):
    edge_index = data.edge_index

    # L_rw = I - D^{-1} A
    lap_edge_index, lap_edge_weight = get_laplacian(edge_index, normalization='rw')

    L = to_dense_adj(lap_edge_index, edge_attr=lap_edge_weight,
                     max_num_nodes=data.num_nodes).squeeze(0)
    return L


def laplacian_positional_encoding(data, k, normalization='sym'):
    """
    Compute top-k non-trivial Laplacian eigenvectors for a PyG Data graph.

    Parameters
    ----------
    data          : torch_geometric.data.Data Graph
                    with data.edge_index and data.num_nodes.
    k             : int Number of *non-trivial* eigenvectors
                    to return (skips the constant one).
    normalization : {'sym', 'rw', None}, optional
          - 'sym' : symmetric normalized Laplacian  L = I - D^{-1/2} A D^{-1/2}
          - 'rw'  : random-walk Laplacian           L = I - D^{-1} A
          - None  : combinatorial Laplacian         L = D - A

    Returns
    -------
    eigvecs : torch.Tensor, shape [num_nodes, k]
              The k Laplacian eigenvectors (columns), as float32.
    eigvals : torch.Tensor, shape [k]
              Corresponding eigenvalues (ascending, skipping the trivial 0).
    """
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    # 1) Build Laplacian in sparse form
    lap_edge_index, lap_edge_weight = get_laplacian(
        edge_index,
        normalization=normalization,
        num_nodes=num_nodes
    )

    L = to_scipy_sparse_matrix(lap_edge_index, lap_edge_weight, num_nodes=num_nodes)

    # 2) Compute smallest (k+1) eigenpairs: include trivial 0, then skip it
    #    (for connected graphs; if graph is disconnected you may want more)
    evals, evecs = eigsh(L, k=k+1, which='SM')  # SM = smallest magnitude

    # 3) Sort just in case and drop the first (trivial) eigenvector
    idx = evals.argsort()
    evals, evecs = evals[idx], evecs[:, idx]

    # drop the first eigenpair (corresponding to eigenvalue ~0)
    evals_nontriv = evals[1:k+1]
    evecs_nontriv = evecs[:, 1:k+1]

    eigvecs = torch.from_numpy(evecs_nontriv).float()
    eigvals = torch.from_numpy(evals_nontriv).float()
    return eigvecs, eigvals


def make_undirected(data):
    data = data.clone()
    data.edge_index = to_undirected(data.edge_index, num_nodes=data.num_nodes)
    return data

In [ ]:
L = combinatorial_laplacian(one_graph)
print(L.shape)
Lambda = laplacian_positional_encoding(one_graph, 10, normalization=None)
print(Lambda[0].shape)
print(Lambda[1])

torch.Size([10230, 10230])
torch.Size([10230, 10])
tensor([0.0141, 0.0166, 0.0182, 0.0183, 0.0212, 0.0246, 0.0272, 0.0562, 0.0599,
        0.0641])


##Confirm the computed eigenvectors satisfy the eigenvector equation. Note we have to convert the graphs to undirected to get a symmetric Laplacian.

In [ ]:
undirected_graph = make_undirected(one_graph)

L = combinatorial_laplacian(undirected_graph)
eigvecs, eigvals = laplacian_positional_encoding(undirected_graph, 10, normalization=None)

i = 2
v = eigvecs[:, i]
lam = eigvals[i]

Lv = L @ v
lamv = lam * v

residual = Lv - lamv
err = residual.norm().item()
rel_err = err / (lamv.norm().item() + 1e-12)

print("‖Lv - λv‖ =", err)
print("relative error =", rel_err)

‖Lv - λv‖ = 3.320720134070143e-05
relative error = 0.5093721702625378


### Compute a new dataset with augmented positional information

In [ ]:
from torch_geometric.data import Data, Dataset

def augment_with_lappe(data: Data, k: int, normalization=None) -> Data:
    """
    Return a clone of `data` where node features are augmented with
    k-dimensional Laplacian positional encodings.
    """
    # Work on an undirected version for symmetric Laplacian
    data_u = make_undirected(data)

    # Compute eigenvectors on the undirected graph
    eigvecs, eigvals = laplacian_positional_encoding(
        data_u, k=k, normalization=normalization
    )  # eigvecs: [N, k_eff]

    # If we requested k but got fewer (small graph), we can pad or just concat as-is.
    # Here we just concat as-is; dimensions will be <= k.
    pe = eigvecs  # [num_nodes, k_eff]

    aug_data = data.clone()

    if aug_data.x is None:
        aug_data.x = pe
    else:
        # Concatenate along feature dimension
        aug_data.x = torch.cat([aug_data.x, pe], dim=-1)

    # Optionally store eigenvalues too if you want them later:
    aug_data.lap_eigvals = eigvals

    return aug_data


def augment_dataset_with_lappe(dataset: Dataset, k: int, normalization=None):
    """
    Iterate through a PyG Dataset, augment each graph with Laplacian PEs,
    and return a list of augmented Data objects.
    """
    augmented_graphs = []

    for data in tqdm(dataset, desc="Augmenting graphs with Laplacian PEs"):
        if data.num_nodes > 5000 or data.num_nodes <= 2*k:
            continue
        aug_data = augment_with_lappe(data, k=k, normalization=normalization)
        augmented_graphs.append(aug_data)

    return augmented_graphs
    '''
    for i, data in enumerate(dataset):
        if data.num_nodes > 5000:
            continue
        print(f"Processing graph {i+1}/{len(dataset)} ...")
        aug_data = augment_with_lappe(data, k=k, normalization=normalization)
        augmented_graphs.append(aug_data)
    return augmented_graphs
    '''




In [ ]:
from torch_geometric.data import Data, Dataset
from torch_geometric.utils import get_laplacian, to_dense_adj, to_scipy_sparse_matrix, to_undirected
from scipy.sparse.linalg import eigsh  # sparse symmetric eigensolver

def make_undirected(data):
    data = data.clone()
    data.edge_index = to_undirected(data.edge_index, num_nodes=data.num_nodes)
    return data

def laplacian_positional_encoding(data, k, normalization='sym'):
    """
    Compute top-k non-trivial Laplacian eigenvectors for a PyG Data graph.

    Parameters
    ----------
    data          : torch_geometric.data.Data Graph
                    with data.edge_index and data.num_nodes.
    k             : int Number of *non-trivial* eigenvectors
                    to return (skips the constant one).
    normalization : {'sym', 'rw', None}, optional
          - 'sym' : symmetric normalized Laplacian  L = I - D^{-1/2} A D^{-1/2}
          - 'rw'  : random-walk Laplacian           L = I - D^{-1} A
          - None  : combinatorial Laplacian         L = D - A

    Returns
    -------
    eigvecs : torch.Tensor, shape [num_nodes, k]
              The k Laplacian eigenvectors (columns), as float32.
    eigvals : torch.Tensor, shape [k]
              Corresponding eigenvalues (ascending, skipping the trivial 0).
    """
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    # 1) Build Laplacian in sparse form
    lap_edge_index, lap_edge_weight = get_laplacian(
        edge_index,
        normalization=normalization,
        num_nodes=num_nodes
    )

    L = to_scipy_sparse_matrix(lap_edge_index, lap_edge_weight, num_nodes=num_nodes)

    # 2) Compute smallest (k+1) eigenpairs: include trivial 0, then skip it
    #    (for connected graphs; if graph is disconnected you may want more)
    nev = k + 1
    # Calculate an appropriate ncv value to prevent ArpackError
    # ncv must be between nev + 1 and num_nodes - 1.
    # A common heuristic is ncv = max(2 * nev + 1, nev + 8).
    ncv_min_req = nev + 1
    ncv_heuristic = max(2 * nev + 1, nev + 8)
    ncv = min(num_nodes - 1, ncv_heuristic)

    # Ensure ncv is at least the minimum required for eigsh to function
    if ncv < ncv_min_req:
        ncv = ncv_min_req

    # Also, ensure ncv does not exceed num_nodes - 1 (or num_nodes in some cases, but num_nodes-1 is safer)
    ncv = min(ncv, num_nodes - 1)

    # If, after all adjustments, ncv is still not greater than nev, eigsh will likely fail.
    # This typically means the graph is too small for the requested number of eigenvectors.
    if ncv <= nev:
        raise ValueError(f"Cannot compute {nev} eigenvectors for graph with {num_nodes} nodes, 'k' might be too large for this graph size. Current k={k}, num_nodes={num_nodes}, ncv={ncv}.")

    evals, evecs = eigsh(L, k=nev, which='SM', ncv=ncv)  # SM = smallest magnitude

    # 3) Sort just in case and drop the first (trivial) eigenvector
    idx = evals.argsort()
    evals, evecs = evals[idx], evecs[:, idx]

    # drop the first eigenpair (corresponding to eigenvalue ~0)
    evals_nontriv = evals[1:k+1]
    evecs_nontriv = evecs[:, 1:k+1]

    eigvecs = torch.from_numpy(evecs_nontriv).float()
    eigvals = torch.from_numpy(evals_nontriv).float()
    return eigvecs, eigvals

def augment_with_lappe(data: Data, k: int, normalization=None) -> Data:
    """
    Return a clone of `data` where node features are augmented with
    k-dimensional Laplacian positional encodings.
    """
    # Work on an undirected version for symmetric Laplacian
    data_u = make_undirected(data)

    # Compute eigenvectors on the undirected graph
    eigvecs, eigvals = laplacian_positional_encoding(
        data_u, k=k, normalization=normalization
    )  # eigvecs: [N, k_eff]

    # If we requested k but got fewer (small graph), we can pad or just concat as-is.
    # Here we just concat as-is; dimensions will be <= k.
    pe = eigvecs  # [num_nodes, k_eff]

    aug_data = data.clone()

    if aug_data.x is None:
        aug_data.x = pe
    else:
        # Concatenate along feature dimension
        aug_data.x = torch.cat([aug_data.x, pe], dim=-1)

    # Optionally store eigenvalues too if you want them later:
    aug_data.lap_eigvals = eigvals

    return aug_data

def augment_dataset_with_lappe(dataset: Dataset, k: int, normalization=None):
    """
    Iterate through a PyG Dataset, augment each graph with Laplacian PEs,
    and return a list of augmented Data objects.
    """
    augmented_graphs = []

    for data in tqdm(dataset, desc="Augmenting graphs with Laplacian PEs"):
        if data.num_nodes > 5000 or data.num_nodes <= 3*k:
            continue
        aug_data = augment_with_lappe(data, k=k, normalization=normalization)
        augmented_graphs.append(aug_data)

    return augmented_graphs


In [ ]:
import pickle

ks = [7, 9]  # You can adjust this value for the number of Laplacian PE dimensions
normalization = None  # Choose 'sym', 'rw', or None for combinatorial Laplacian

for k in ks:
  # Call augment_dataset_with_lappe to process the dataset
  augmented_graphs = augment_dataset_with_lappe(data, k=k, normalization=normalization)

  # Define the output path for the pickle file
  output_path = f"dataset_with_lapPE{k}.pkl"

  # Save the augmented dataset as a pickle file
  with open(output_path, "wb") as f:
      pickle.dump(augmented_graphs, f)

  print(f"Saved augmented dataset with Laplacian PEs to: {output_path}")

Augmenting graphs with Laplacian PEs: 100%|██████████| 27085/27085 [34:42<00:00, 13.01it/s]


Saved augmented dataset with Laplacian PEs to: dataset_with_lapPE7.pkl


Augmenting graphs with Laplacian PEs: 100%|██████████| 27085/27085 [36:06<00:00, 12.50it/s]


Saved augmented dataset with Laplacian PEs to: dataset_with_lapPE9.pkl


In [ ]:
from torch_geometric.utils import get_laplacian, to_dense_adj, to_scipy_sparse_matrix, to_undirected
from scipy.sparse.linalg import eigsh  # sparse symmetric eigensolver

def laplacian_positional_encoding(data, k, normalization='sym'):
    """
    Compute top-k non-trivial Laplacian eigenvectors for a PyG Data graph.

    Parameters
    ----------
    data          : torch_geometric.data.Data Graph
                    with data.edge_index and data.num_nodes.
    k             : int Number of *non-trivial* eigenvectors
                    to return (skips the constant one).
    normalization : {'sym', 'rw', None}, optional
          - 'sym' : symmetric normalized Laplacian  L = I - D^{-1/2} A D^{-1/2}
          - 'rw'  : random-walk Laplacian           L = I - D^{-1} A
          - None  : combinatorial Laplacian         L = D - A

    Returns
    -------
    eigvecs : torch.Tensor, shape [num_nodes, k]
              The k Laplacian eigenvectors (columns), as float32.
    eigvals : torch.Tensor, shape [k]
              Corresponding eigenvalues (ascending, skipping the trivial 0).
    """
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    # 1) Build Laplacian in sparse form
    lap_edge_index, lap_edge_weight = get_laplacian(
        edge_index,
        normalization=normalization,
        num_nodes=num_nodes
    )

    L = to_scipy_sparse_matrix(lap_edge_index, lap_edge_weight, num_nodes=num_nodes)

    # 2) Compute smallest (k+1) eigenpairs: include trivial 0, then skip it
    #    (for connected graphs; if graph is disconnected you may want more)
    nev = k + 1
    # Calculate an appropriate ncv value to prevent ArpackError
    # ncv must be between nev + 1 and num_nodes - 1.
    # A common heuristic is ncv = max(2 * nev + 1, nev + 8).
    ncv_min_req = nev + 1
    ncv_heuristic = max(2 * nev + 1, nev + 8)
    ncv = min(num_nodes - 1, ncv_heuristic)

    # Ensure ncv is at least the minimum required for eigsh to function
    if ncv < ncv_min_req:
        ncv = ncv_min_req

    # Also, ensure ncv does not exceed num_nodes - 1 (or num_nodes in some cases, but num_nodes-1 is safer)
    ncv = min(ncv, num_nodes - 1)

    # If, after all adjustments, ncv is still not greater than nev, eigsh will likely fail.
    # This typically means the graph is too small for the requested number of eigenvectors.
    if ncv <= nev:
        raise ValueError(f"Cannot compute {nev} eigenvectors for graph with {num_nodes} nodes, 'k' might be too large for this graph size. Current k={k}, num_nodes={num_nodes}, ncv={ncv}.")

    evals, evecs = eigsh(L, k=nev, which='SM', ncv=ncv)  # SM = smallest magnitude

    # 3) Sort just in case and drop the first (trivial) eigenvector
    idx = evals.argsort()
    evals, evecs = evals[idx], evecs[:, idx]

    # drop the first eigenpair (corresponding to eigenvalue ~0)
    evals_nontriv = evals[1:k+1]
    evecs_nontriv = evecs[:, 1:k+1]

    eigvecs = torch.from_numpy(evecs_nontriv).float()
    eigvals = torch.from_numpy(evals_nontriv).float()
    return eigvecs, eigvals


In [ ]:
aug_data = augment_with_lappe(data[6], k=8, normalization=None)
print(aug_data)

KeyboardInterrupt: 

In [ ]:
print("")
print("number of graphs: ", len(augmented_graphs))
print('==============================================================')
idx = 4
one_graph = augmented_graphs[idx]  # Get the first graph object.

# Gather some statistics about the graph.
print("graph at index: ", idx)
print(one_graph)
print(f'Number of nodes: {one_graph.num_nodes}')
print(f'Number of edges: {one_graph.num_edges}')
print(f'Average node degree: {(one_graph.num_edges) / one_graph.num_nodes:.2f}')
print(f'Contains isolated nodes: {one_graph.has_isolated_nodes()}')
print(f'Contains self-loops: {one_graph.has_self_loops()}')
print(f'Is undirected: {one_graph.is_undirected()}')


number of graphs:  26877
graph at index:  4
Data(x=[580, 51], edge_index=[2, 579], edge_attr=[579], y=[41], lap_eigvals=[8])
Number of nodes: 580
Number of edges: 579
Average node degree: 1.00
Contains isolated nodes: False
Contains self-loops: False
Is undirected: False
